Quantified backlog state = Sum(Rule \* Weight \* 10)

This will give the team clear information about backlog state score in a number from 1-10 (1 - worst, 10 - best )
Weights need to sum up to 100%



In [ ]:
from jira import JIRA
import pandas as pd
import re
from numpy import nan
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:


#to show all rules descriptions
pd.set_option('max_colwidth',200)

rules = pd.DataFrame()
rules['rule'] = ''
rules['weight'] = 0
rules['value'] = 0

rules['rule'] = ['In next 2 sprints there are items in Backlog state that SP sum is greater than estimated velocity, there are no items assigned to sprint that are not in Backlog state - Y/N',
                'Planned next 3 versions (all items estimated and in Backlog state) - Y/N',
                '% Should, Must, Critical items that are estimated regardless of status',
                '% of key milestone items estimated and in Backlog status']
rules['weight'] = [0.4, 0.25, 0.25, 0.1]

rules

Rules sum should be 1

In [ ]:
sum(rules.weight)

<div class="alert alert-block alert-success">
Set up nextSprint, currentVersion, nextVersion values.

They can be loaded automatically from Jira too.
</div>

In [ ]:
nextSprints = ['IC - #4 20180219 - 1.13', 'IC - #5 20180305 - 1.14']

#velocity used to check if sprints are planned correctly
estimatedVelocity = 13

nextVersions = ['1.12', '1.13', '1.14', '1.15', '1.16']
milestones = ['Frimley MVP']

In [ ]:

jira_url = 'https://kainos-evolve.atlassian.net'
jira = JIRA(jira_url)

In [ ]:
#rule 0
jql = 'sprint in ("{}") and status = Backlog and type not in subTaskIssueTypes()'.format('", "'.join(nextSprints))
jql

issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['sprint'] = ''
issues['SP'] = ''
issues['type'] = ''
issues['status'] = ''
issues['summary'] = ''

for issue in issuesRaw:
    for rawSprint in issue.fields.customfield_10007:
        #unfortunately sprint information is encoded and regex is needed
        matches = re.search('name=(.*?),', rawSprint)
        issues = issues.append(
            {
             'key': issue.key,
             'type': issue.fields.issuetype.name,
             'status': issue.fields.status.name,
             'SP' : issue.fields.customfield_10005,
             'summary': issue.fields.summary,
             'sprint' : matches.group(1)
            }, ignore_index=True)

issues.fillna(value=nan, inplace=True)
#issues.fillna(0, inplace=True)
issues

sprints = issues.groupby(['sprint']).agg({'SP':'sum'})
sprints = sprints.loc[(sprints['SP'] >= estimatedVelocity)]
sprints
rule_value = (len(sprints) == 2)
rules.at[0, 'value'] = int(rule_value)

In [ ]:
#rule 1 Planned next 3 versions (all items estimated and in Backlog state) - Y/N
jql = 'fixVersion in ("{}") and status = Backlog and type not in subTaskIssueTypes()'.format('", "'.join(nextVersions))
jql

issuesRaw = jira.search_issues(jql)

issues = pd.DataFrame()
issues['key'] = ''
issues['version'] = ''
issues['SP'] = ''
issues['type'] = ''
issues['status'] = ''
issues['summary'] = ''

#add issues to dataframe
for issue in issuesRaw:
    #issue may have many versions - in this approach, one version per issue is recommended
    for fixVersion in issue.fields.fixVersions:
        if(fixVersion.name in nextVersions):
            issues = issues.append(
                {'version': fixVersion.name, 
                 'key': issue.key,
                 'type': issue.fields.issuetype.name,
                 'status': issue.fields.status.name,
                 'SP': issue.fields.customfield_10005,
                 'summary': issue.fields.summary,
                 'team' : str(issue.fields.customfield_14200),
                }, ignore_index=True)


issues['isEstimated'] = False
issues.loc[(issues['type'] == 'Bug'), ['isEstimated']] = True
issues.loc[(issues['type'] != 'Bug') & ~(issues['SP'].isnull()), ['isEstimated']] = True
issues.sort_values("version", inplace=True)
issues

print('Issues in versions')
issuesInVersion = issues.groupby(['version']).agg({'key':'count'})
issuesInVersion.columns = ['count']
issuesInVersion = issuesInVersion.reset_index()
issuesInVersion

print('Not estimated issues in versions')
notEstimatedIssuesInVersion = issues.loc[~(issues['isEstimated'])].groupby(['version']).agg({'key':'count'})
notEstimatedIssuesInVersion.columns = ['count']
notEstimatedIssuesInVersion = notEstimatedIssuesInVersion.reset_index()
notEstimatedIssuesInVersion

allEstimated = True

for version in nextVersions:
    itemsCount = len(issuesInVersion.loc[(issuesInVersion['version'] == version)])
    notEstimatedItemsCount = len(notEstimatedIssuesInVersion.loc[(notEstimatedIssuesInVersion['version'] == version)])
    allEstimated = itemsCount > 0 and notEstimatedItemsCount == 0

rules.at[1, 'value'] = int(allEstimated)

In [ ]:
rules